In [5]:
#ollama model과 HuggingFaceHub 모델을 사용하여 한국어로 질문에 답변하는 챗봇 예제
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain.memory import ConversationBufferWindowMemory
from dotenv import load_dotenv
import requests
import json
from langchain_community.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings
load_dotenv()

# Ollama
ollama_emb = OllamaEmbeddings(model="nomic-embed-text")
vec1 = ollama_emb.embed_query("올라마 임베딩 예시")

# Hugging Face
hf_emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vec2 = hf_emb.embed_query("허깅페이스 임베딩 예시")

print(len(vec1), len(vec2))



# 전역 메모리 객체
memory = ConversationBufferWindowMemory(
    k=5,  # 최근 5개의 대화만 기억
    return_messages=True,
    memory_key="chat_history"
)

768 384


In [6]:
def load_and_process_documents():
    #문서 로드 및 분할
    print("문서 로드하는 중...")
    
    # 샘플 문서
    documents = [
        Document(page_content="LangChain은 파이썬 기반의 LLM 프레임워크입니다. 다양한 AI 모델과 연동할 수 있습니다."),
        Document(page_content="LCEL은 LangChain Expression Language의 줄임말로, 복잡한 체인을 쉽게 구성할 수 있게 해주는 언어입니다."),
        Document(page_content="RAG는 Retrieval Augmented Generation의 줄임말로, 외부 문서를 검색하여 답변의 정확성을 높이는 기법입니다."),
        Document(page_content="벡터 데이터베이스는 텍스트를 벡터로 변환하여 의미론적 유사성을 기반으로 검색할 수 있게 해줍니다."),
        Document(page_content="프롬프트 엔지니어링은 AI 모델에게 더 나은 응답을 얻기 위해 입력을 최적화하는 기법입니다.")
    ]
    
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    
    splits = text_splitter.split_documents(documents)
    print(f"문서를 {len(splits)}개의 청크로 분할했습니다.")
    
    return splits


In [132]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def create_vector_store(documents):
    #벡터 스토어 생성
    print("벡터 스토어 생성 중...")
    
    query = "LangChain과 LCEL, RAG가 무엇인가요?"
    # Ollama
    ollama_emb = OllamaEmbeddings(model="nomic-embed-text")

    # Hugging Face
    hf_emb = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")
    print("---")
    print("OLLAMA 벡터 스토어 생성 완료!")
    print("---")
    vector_store_ol = FAISS.from_documents(documents=documents, embedding=ollama_emb)


    qvec1 = np.array(ollama_emb.embed_query(query)).reshape(1, -1)

    # 모든 문서 벡터 꺼내기
    doc_vec1 = np.vstack([vector_store_ol.index.reconstruct(i) for i in range(vector_store_ol.index.ntotal)])
    sim1 = cosine_similarity(qvec1, doc_vec1)[0]

    # 상위 3개만 추출
    top_idx1 = sim1.argsort()[::-1][:3]
    print("Example query:", query)
    print("---")
    for i, idx in enumerate(top_idx1, 1):
        doc_id = vector_store_ol.index_to_docstore_id[idx]
        doc = vector_store_ol.docstore.search(doc_id)
        print(f"#{i} Cosine similarity: {sim1[idx]:.4f}")
        print(f"#{i} 문서: {doc.page_content[:100]}...")
        print("---")

    #print("---")
    print("Hugging face 벡터 스토어 생성 완료!")
    print("---")
    vector_store_hf = FAISS.from_documents(documents=documents, embedding=hf_emb)
    qvec2 = np.array(hf_emb.embed_query(str(query))).reshape(1, -1)

    # 모든 문서 벡터 꺼내기
    doc_vec2 = np.vstack([vector_store_hf.index.reconstruct(i) for i in range(vector_store_hf.index.ntotal)])
    sim2 = cosine_similarity(qvec2, doc_vec2)[0]

    # 상위 3개만 추출
    top_idx2 = sim2.argsort()[::-1][:3]
    print("Example query:", query)
    print("---")

    for i, idx in enumerate(top_idx2, 1):
        doc_id = vector_store_hf.index_to_docstore_id[idx]
        doc = vector_store_hf.docstore.search(doc_id)
        print(f"#{i} Cosine similarity: {sim2[idx]:.4f}")
        print(f"#{i} 문서: {doc.page_content[:100]}...")
        print("---")
    
    return vector_store_ol, vector_store_hf

def format_docs(docs):
    #검색된 문서들을 문자열로 포맷팅
    return "\n\n--- 문서 내용 ---\n\n".join([doc.page_content for doc in docs])

In [133]:
def get_chat_history():
    #채팅 히스토리를 문자열로 변환
    messages = memory.chat_memory.messages
    if not messages:
        return "이전 대화 기록이 없습니다."
    
    history_str = []
    for msg in messages[-6:]:  # 최근 3턴만
        if hasattr(msg, 'type'):
            if msg.type == 'human':
                history_str.append(f"사용자: {msg.content}")
            elif msg.type == 'ai':
                history_str.append(f"AI: {msg.content}")
    
    return "\n".join(history_str) if history_str else "이전 대화 기록이 없습니다."

In [134]:
def create_rag_chain(vector_store):
    #LCEL을 사용한 RAG 체인 생성
    
    # LLM 초기화
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.3,
        max_tokens=1000
    )
    
    # 검색기 설정
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    # 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_template("""
당신은 문서의 내용을 바탕으로 질문에 답변하는 도우미입니다.
주어진 문맥 정보와 이전 대화 기록을 모두 고려하여 정확하고 도움이 되는 답변을 한글로 제공해주세요.

이전 대화 기록:
{chat_history}

관련 문서 내용:
{context}

현재 질문: {question}

답변 시 다음 사항을 고려해주세요:
- 문서의 내용을 정확하게 반영하여 답변하세요
- 이전 대화의 맥락을 고려하여 답변하세요
- 구체적인 예시나 데이터가 있다면 포함해주세요
- 답변은 한국어로 명확하고 이해하기 쉽게 작성해주세요
- 문서에 없는 내용은 추측하지 마세요
- 답변은 500자 이내로 간결하게 작성해주세요

답변:
""")
    
    # LCEL 체인 구성
    chain = (
        RunnableParallel({
            "context": retriever | format_docs,
            "question": RunnablePassthrough(),
            "chat_history": lambda x: get_chat_history()
        })
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [135]:
def ask_question(chain, question):
    #질문하고 답변받기 (메모리 업데이트 포함)
    print(f"\n질문: {question}")

    # 체인 실행
    response = chain.invoke(question)
        
    # 메모리에 대화 기록 저장
    memory.chat_memory.add_user_message(question)
    memory.chat_memory.add_ai_message(response)
        
    print(f"답변: {response}")
    print(f"[메모리] 저장된 대화 수: {len(memory.chat_memory.messages)//2}턴")
        
    return response

In [140]:
def show_memory():
    #메모리 내용 출력
    messages = memory.chat_memory.messages
    
    if not messages:
        print("저장된 대화가 없습니다.")
        return
    
    for i, msg in enumerate(messages):
        if hasattr(msg, 'type'):
            if msg.type == 'human':
                print(f"#{i//2 + 1} 질문: {msg.content}")
            elif msg.type == 'ai':
                print(f"#{i//2 + 1} 답변: {msg.content[:100]}...")
    print("=" * 50)

In [137]:
# 메인 실행 부분
if __name__ == "__main__":
    # 1. 문서 로드 및 처리
    documents = load_and_process_documents()
    
    # 2. 벡터 스토어 생성
    vector_store_ol, vector_store_hf = create_vector_store(documents)
    
    # 3. RAG 체인 생성
    rag_chain_ol = create_rag_chain(vector_store_ol)
    rag_chain_hf = create_rag_chain(vector_store_hf)

문서 로드하는 중...
문서를 5개의 청크로 분할했습니다.
벡터 스토어 생성 중...
---
OLLAMA 벡터 스토어 생성 완료!
---
Example query: LangChain과 LCEL, RAG가 무엇인가요?
---
#1 Cosine similarity: 0.7687
#1 문서: LCEL은 LangChain Expression Language의 줄임말로, 복잡한 체인을 쉽게 구성할 수 있게 해주는 언어입니다....
---
#2 Cosine similarity: 0.7259
#2 문서: LangChain은 파이썬 기반의 LLM 프레임워크입니다. 다양한 AI 모델과 연동할 수 있습니다....
---
#3 Cosine similarity: 0.6068
#3 문서: RAG는 Retrieval Augmented Generation의 줄임말로, 외부 문서를 검색하여 답변의 정확성을 높이는 기법입니다....
---
Hugging face 벡터 스토어 생성 완료!
---
Example query: LangChain과 LCEL, RAG가 무엇인가요?
---
#1 Cosine similarity: 0.7397
#1 문서: LCEL은 LangChain Expression Language의 줄임말로, 복잡한 체인을 쉽게 구성할 수 있게 해주는 언어입니다....
---
#2 Cosine similarity: 0.6595
#2 문서: LangChain은 파이썬 기반의 LLM 프레임워크입니다. 다양한 AI 모델과 연동할 수 있습니다....
---
#3 Cosine similarity: 0.4327
#3 문서: 프롬프트 엔지니어링은 AI 모델에게 더 나은 응답을 얻기 위해 입력을 최적화하는 기법입니다....
---


In [141]:
print("\n" + "="*60)
print("OLLAMA Text Embedding | LCEL 기반 Multi-Turn RAG 시스템 준비완료.")
print("="*60)
    
# 4. Multi-Turn 대화 예시
conversation_flow = [
    "LangChain과 LCEL, RAG가 무엇인가요?",
    "지금까지 설명한 기술들을 어떻게 함께 사용할 수 있나요?"
]
    
for i, question in enumerate(conversation_flow, 1):
    print(f"\n{'='*20} 턴 {i} {'='*20}")
    ask_question(rag_chain_ol, question)
        
    if i == 3:  # 3턴 후 메모리 상태 확인
        show_memory()
    
# 5. 최종 메모리 상태
print(f"\n{'='*20} 최종 메모리 상태 {'='*20}")
show_memory()
memory.clear()  # 메모리 초기화


OLLAMA Text Embedding | LCEL 기반 Multi-Turn RAG 시스템 준비완료.

==================== 턴 1 ====================

질문: LangChain과 LCEL, RAG가 무엇인가요?
답변: LangChain은 파이썬 기반의 LLM(대형 언어 모델) 프레임워크로, 다양한 AI 모델과 연동하여 사용할 수 있는 플랫폼입니다. LCEL은 LangChain Expression Language의 줄임말로, 복잡한 체인을 쉽게 구성할 수 있도록 도와주는 언어입니다. RAG는 Retrieval Augmented Generation의 약자로, 외부 문서를 검색하여 답변의 정확성을 높이는 기법입니다. 이 세 가지 요소는 함께 작동하여 AI 모델의 성능을 향상시키고, 사용자에게 더 정확하고 유용한 정보를 제공하는 데 기여합니다.
[메모리] 저장된 대화 수: 1턴

==================== 턴 2 ====================

질문: 지금까지 설명한 기술들을 어떻게 함께 사용할 수 있나요?
답변: 지금까지 설명한 기술들은 함께 사용하여 AI 모델의 성능을 극대화할 수 있습니다. 예를 들어, LangChain을 활용하여 다양한 AI 모델을 통합하고, LCEL을 사용해 복잡한 체인을 쉽게 구성할 수 있습니다. 이 과정에서 프롬프트 엔지니어링 기법을 적용하여 입력을 최적화하면 AI 모델이 더 나은 응답을 생성할 수 있습니다. 또한, RAG 기법을 통해 외부 문서를 검색하여 답변의 정확성을 높일 수 있습니다. 마지막으로, 벡터 데이터베이스를 활용하여 텍스트를 벡터로 변환하고 의미론적 유사성을 기반으로 검색함으로써, 더욱 정교하고 관련성 높은 정보를 제공할 수 있습니다. 이러한 기술들이 결합되면, 사용자에게 더 유용하고 정확한 결과를 제공할 수 있습니다.
[메모리] 저장된 대화 수: 2턴

==================== 최종 메모리 상태 ====================
#1 질문: LangC

In [142]:
print("\n" + "="*60)
print("Hugging Face Text Embedding | LCEL 기반 Multi-Turn RAG 시스템 준비완료.")
print("="*60)
    
# 4. Multi-Turn 대화 예시
conversation_flow = [
    "LangChain과 LCEL, RAG가 무엇인가요?",
    "지금까지 설명한 기술들을 어떻게 함께 사용할 수 있나요?"
]
    
for i, question in enumerate(conversation_flow, 1):
    print(f"\n{'='*20} 턴 {i} {'='*20}")
    ask_question(rag_chain_hf, question)
        
    if i == 3:  # 3턴 후 메모리 상태 확인
        show_memory()
    
# 5. 최종 메모리 상태
print(f"\n{'='*20} 최종 메모리 상태 {'='*20}")
show_memory()
memory.clear()  # 메모리 초기화


Hugging Face Text Embedding | LCEL 기반 Multi-Turn RAG 시스템 준비완료.

==================== 턴 1 ====================

질문: LangChain과 LCEL, RAG가 무엇인가요?
답변: LangChain은 파이썬 기반의 LLM(대형 언어 모델) 프레임워크로, 다양한 AI 모델과 연동하여 복잡한 작업을 수행할 수 있도록 돕습니다. LCEL은 LangChain Expression Language의 약자로, 복잡한 체인을 쉽게 구성할 수 있게 해주는 언어입니다. 이를 통해 사용자는 더 효율적으로 AI 모델을 활용할 수 있습니다. 

RAG(복합적 응답 생성)는 문서에 명시되어 있지 않지만, 일반적으로 정보 검색과 생성 모델을 결합하여 더 정확하고 유용한 응답을 생성하는 기법을 의미합니다. 이러한 기법은 프롬프트 엔지니어링과 함께 사용되어 AI 모델의 응답 품질을 향상시키는 데 기여합니다.
[메모리] 저장된 대화 수: 1턴

==================== 턴 2 ====================

질문: 지금까지 설명한 기술들을 어떻게 함께 사용할 수 있나요?
답변: LangChain, LCEL, RAG, 벡터 데이터베이스, 그리고 프롬프트 엔지니어링은 함께 사용하여 AI 모델의 성능을 극대화할 수 있습니다. 

예를 들어, LangChain을 사용하여 다양한 AI 모델과 연동하고, LCEL을 통해 복잡한 체인을 구성할 수 있습니다. 이 과정에서 벡터 데이터베이스를 활용하여 텍스트를 벡터로 변환하고, 의미론적 유사성을 기반으로 관련 정보를 검색합니다. 

그 후, 프롬프트 엔지니어링 기법을 적용하여 입력을 최적화함으로써 AI 모델이 더 나은 응답을 생성하도록 유도할 수 있습니다. RAG 기법을 통해 정보 검색과 생성 모델을 결합하면, 더욱 정확하고 유용한 응답을 얻을 수 있습니다. 이러한 통합적인 접근 방식은 AI의 활용도를 높이는 데 기여합니다.
[메모리] 저장된 대화 수: 